<a href="https://colab.research.google.com/github/gabrielelt/Curso_Git/blob/main/Trabalho3_pipeline_inf250_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabalho 3 RiscV Pipeline - Prazo Parte I 21/1 e Parte II 28/1

## Parte I: Python sem forward, hazard e desvio 4 estágio

## Parte II: Adaptar o Simulador Verilog [sem forward](https://colab.research.google.com/drive/1HNi4zRXolKYMk5xypbX0iq8ZJeE4Rn1C?usp=sharing) mas com a animacao do desenho SVG para as versões:
   * **Já está feito** Como Exemplo, está disponível a adaptação do "Com Forward e Harzard, desvio no 4 estágio"[Sem Interface](https://colab.research.google.com/drive/1EziU5Hb609vjwyscSDsbGPhs4gkM3OdC?usp=sharing), primeiro completei o [desenho svg](https://github.com/arduinoufv/inf250/blob/master/2024/pipe_para_completar_forward_hazard.svg) e depois gerei um novo [colab com a versão com Forward e Hazard de Dados](https://colab.research.google.com/drive/1DrAfdXnsjWtteirTV6W7y1dJvUrfuR63?usp=sharing)
   * [Com desvio no segundo estágio](https://colab.research.google.com/drive/1rFd3QLmpMDGUaREZyWzFinT89FATqkgQ?usp=sharing) , importante é completar o [desenho svg](https://github.com/arduinoufv/inf250/blob/master/2024/pipe_para_completar_branch_2st_forward.svg)

# Parte I

* Funcionalidades
    * Ter uma janela para digitar o código
    * Ter no mínimo 5 códigos de exemplo (desenho da tabela de execução) que mostrem as situações de Stall e a capacidade do simulador de tratar corretamente
    * Código Python deve ser de fácil entendimento e possibilidades de extensão
    * **Fetch** deve ter a capacidade de tratar desvio em conjunto com o **Memória**, já que o desvio executa no quarto estágio
    * **Decode** deve ler os registradores e registrar as futuras dependências, os **stalls** devem acontecer no **decode**. O decode também fazer o **parser** da instrução.
    * **Execute** irá executar e deve ser facil de ser estendido para incorporar outras instruções.
    * **Memória** deve gravar usando de 4 em 4 bytes, desconsiderando os 2 bits menos significativos. O desvio ocorre no 4 estágio, quando desvia tem que dar flash na pipeline.
    * **


In [ ]:
#@title Exemplo de Simulador (incompleto)
import ipywidgets as widgets
from IPython.display import display, clear_output

# Configuração do pipeline
stages = ["IF", "ID", "EX", "MEM", "WB"]
program = [
    "add x1, x2, x3",  # x1 = x2 + x3
    "sub x4, x1, x5",  # x4 = x1 - x5
    "mul x6, x4, x7",  # x6 = x4 * x7
    "add x8, x6, x1"   # x8 = x6 + x1
]
registers = {f"x{i}": 0 for i in range(32)}  # Registradores do RISC-V
registers.update({"x2": 10, "x3": 5, "x5": 3, "x7": 2})  # Valores iniciais

# Barreiras (buffers intermediários)
barriers = {
    "IF_ID": None,
    "ID_EX": None,
    "EX_MEM": None,
    "MEM_WB": None
}

# Estágios do pipeline
def fetch():
    if program:
        return program.pop(0)
    return None

def decode(instruction):
    if instruction:
        try:
            parts = instruction.split(maxsplit=1)  # Divide em opcode e argumentos
            op = parts[0]
            args = [arg.strip() for arg in parts[1].split(",")]  # Remove espaços e vírgulas
            if len(args) != 3:
                raise ValueError("Instrução mal formada.")
            return {"op": op, "rd": args[0], "rs1": args[1], "rs2": args[2]}
        except (IndexError, ValueError) as e:
            print(f"Erro ao decodificar instrução: {instruction}. Detalhes: {e}")
            return None
    return None

def execute(decoded_instr):
    if decoded_instr:
        op = decoded_instr["op"]
        rs1 = registers.get(decoded_instr["rs1"], 0)
        rs2 = registers.get(decoded_instr["rs2"], 0)
        if op == "add":
            result = rs1 + rs2
        elif op == "sub":
            result = rs1 - rs2
        elif op == "mul":
            result = rs1 * rs2
        else:
            result = None
        return {"rd": decoded_instr["rd"], "result": result}
    return None

def memory(access):
    # Para este exemplo, não simulamos memória externa
    return access

def write_back(wb):
    if wb:
        registers[wb["rd"]] = wb["result"]

# Avançar o ciclo do pipeline
def pipeline_cycle():
    global barriers
    # Executar estágios na ordem inversa
    write_back(barriers["MEM_WB"])
    barriers["MEM_WB"] = memory(barriers["EX_MEM"])
    barriers["EX_MEM"] = execute(barriers["ID_EX"])
    barriers["ID_EX"] = decode(barriers["IF_ID"])
    barriers["IF_ID"] = fetch()

# Mostrar estado do pipeline
def display_pipeline(cycle):
    clear_output(wait=True)
    print(f"Ciclo {cycle}:")
    print(f"Registradores: {registers}")
    for barrier, value in barriers.items():
        print(f"{barrier}: {value}")
    print("-" * 40)

    # Reexibir os botões para manter o estado da interface
    display(widgets.VBox([cycle_label, clock_button]))

# Interface com botão para simulação
cycle = 0
cycle_label = widgets.Label(value=f"Ciclo: {cycle}")
clock_button = widgets.Button(description="Clock")

def on_clock_button_click(b):
    global cycle
    pipeline_cycle()
    cycle += 1
    cycle_label.value = f"Ciclo: {cycle}"
    display_pipeline(cycle)

clock_button.on_click(on_clock_button_click)

# Inicializar interface
display(widgets.VBox([cycle_label, clock_button]))
display_pipeline(cycle)


Ciclo 0:
Registradores: {'x0': 0, 'x1': 0, 'x2': 10, 'x3': 5, 'x4': 0, 'x5': 3, 'x6': 0, 'x7': 2, 'x8': 0, 'x9': 0, 'x10': 0, 'x11': 0, 'x12': 0, 'x13': 0, 'x14': 0, 'x15': 0, 'x16': 0, 'x17': 0, 'x18': 0, 'x19': 0, 'x20': 0, 'x21': 0, 'x22': 0, 'x23': 0, 'x24': 0, 'x25': 0, 'x26': 0, 'x27': 0, 'x28': 0, 'x29': 0, 'x30': 0, 'x31': 0}
IF_ID: None
ID_EX: None
EX_MEM: None
MEM_WB: None
----------------------------------------


## versão incompleta

* Não tem memória, só banco de registradores
* **fetch** retira da lista de instruções e não trata desvios
* **decode** apenas "separa" os campos mas não lê valores nem toma ações
* **execute** faz a leitura dos registradores que deve ser no **decode**, apenas executa alguns exemplos de instruções tipo R de ALU
* **memory** não faz nada ainda....
* **write back** atualiza os valores

# Parte II
